## Import Modules

In [1]:
import numpy as np
import pandas as pd

## Load Data

In [2]:
dtype = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('../week-4/kc_house_data.csv', dtype = dtype)
train = pd.read_csv('../week-4/kc_house_train_data.csv', dtype = dtype)
test = pd.read_csv('../week-4/kc_house_test_data.csv', dtype = dtype)

Next, from Module 2 (Multiple Regression), copy and paste the `get_numpy_data` function (or equivalent) that takes a data set, a list of features (e.g. [‘sqft_living’, ‘bedrooms’]), to be used as inputs, and a name of the output (e.g. `price`). This function returns a `feature_matrix` (2D array) consisting of first a column of ones followed by columns containing the values of the input features in the data set in the same order as the input list. It also returns an `output_array` which is an array of the values of the output in the data set (e.g. `price`).

In [3]:
def get_numpy_data(
        # data_sframe, 
        data_frame, features, output):
    # data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    data_frame = data_frame.assign(constant = 1)
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    feature_df = data_frame.loc[:, features]
    # the following line will convert the features_SFrame into a numpy matrix:
    # feature_matrix = features_sframe.to_numpy()
    feature_matrix = feature_df.values
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_df = data_frame.loc[:, output]
    # the following will convert the SArray into a numpy array by first converting it to a list
    # output_array = output_sarray.to_numpy()
    output_array = output_df.values
    
    return(feature_matrix, output_array)

Similarly, copy and paste the `predict_output` function (or equivalent) from Module 2. This function accepts a 2D array `feature_matrix` and a 1D array `weights` and return a 1D array `predictions`.

In [4]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because “small” weights are dropped first as `l1_penalty` goes up.

To give equal considerations for all features, we need to normalize features as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data.

**Hint**: If you are using Numpy, a handy shorthand for computing 2-norms of columns is

`norms = np.linalg.norm(X, axis=0)`

To normalize columns, perform element-wise division.

`X_normalized = X / norms`

In [5]:
def normalize_features(features):
    norms = np.linalg.norm(features, axis = 0)
    normalized_features = features / norms
    return (normalized_features, norms)

## Review of Coordinate Descent

 We seek to obtain a sparse set of weights by minimizing the LASSO cost function
 
`SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`

(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use coordinate descent: at each iteration, we will fix all weights but `weight i` and find the value of `weight i` that minimizes the objective. That is, we look for

`argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]`

where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.

* Pick a coordinate i
* Compute `w[i]` that minimizes the LASSO cost function
* Repeat the two steps for all coordinates, multiple times

For this assignment, we use cyclical coordinate descent with normalized features, where we cycle through coordinates `0` to `(d-1)` in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:

       ┌ (ro[i] + lambda/2)    if ro[i] < -lambda/2
       
w[i] = ├ 0                     if -lambda/2 <= ro[i] <= lambda/2

       └ (ro[i] - lambda/2)    if ro[i] > lambda/2
       
where

`ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]`

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:

`w[0] = ro[i]`

## Effect of L1 penalty

Consider a simple model with 2 features: `sqft_living` and `bedrooms`. The output is `price`.

* First, run `get_numpy_data()` (or equivalent) to obtain a feature matrix with 3 columns (constant column added). Use the entire `sales` dataset for now.
* Normalize columns of the feature matrix. Save the norms of original features as `norms`.
* Set initial weights to `[1,4,1]`.
* Make predictions with feature matrix and initial weights.
* Compute values of `ro[i]`, where

`ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]`

**Quiz Question 1**: Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is set to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` would not set`w[1]` zero, but would set `w[2]` to zero, if we were to take a step in that coordinate?

In [6]:
# run get_numpy_data
feature_matrix, output_array = get_numpy_data(sales, ['sqft_living', 'bedrooms'], ['price'])

In [7]:
# normalize columns
normalized_features, norms = normalize_features(feature_matrix)

In [8]:
# set initial weights
weights = [1, 4, 1]

In [9]:
# make predictions
predictions = predict_output(feature_matrix, weights)

In [10]:
# compute value of ro[i]
ro = []

import time
start_time = time.time()

ro.append(np.sum(normalized_features[:, 0] * (output_array[:, 0] - predictions + weights[0] * normalized_features[:, 0])))
ro.append(np.sum(normalized_features[:, 1] * (output_array[:, 0] - predictions + weights[1] * normalized_features[:, 1])))
ro.append(np.sum(normalized_features[:, 2] * (output_array[:, 0] - predictions + weights[2] * normalized_features[:, 2])))

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.0017290115356445312 seconds ---


In [11]:
ro

[78176568.83295017, 86601827.9592994, 79704172.60800107]

In [12]:
from decimal import Decimal

print('ro[0]: %.2E' % ro[0])
print('ro[1]: %.2E' % ro[1])
print('ro[2]: %.2E' % ro[2])

ro[0]: 7.82E+07
ro[1]: 8.66E+07
ro[2]: 7.97E+07


In [13]:
l1_penalty_min_qz1 = 2 * ro[2]
l1_penalty_max_qz1 = 2 * ro[1]  # exclusive

print('range is [%.2E, %.3E)'% (l1_penalty_min_qz1, l1_penalty_max_qz1) )

range is [1.59E+08, 1.732E+08)


**Quiz Question 2**: What range of values of `l1_penalty` would set both w[1] and w[2] to zero, if we were to take a step in that coordinate?

In [14]:
l1_penalty_min_qz2 = 2 * ro[1]

print('range is [%.3E, infinity)' % (l1_penalty_min_qz2))

range is [1.732E+08, infinity)


So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

e.g. in Python:

In [15]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    """
    : type i: integer
    : type feature_matrix: 2d numpy array
    : type output: 1d numpy array
    : type weights: 1d numpy array
    : type l1_penalty: float
    : rtype new_weight_i: float
    """
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    ro_i = np.sum(feature_matrix[:, i] * (output - prediction + weights[i] * feature_matrix[:, i]))
    
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2.
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2.
    else:
        new_weight_i = 0.
    
    return new_weight_i

If you are using Numpy, test your function with the following snippet:

In [16]:
# should print 0.425558846691
import math
print(lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],
                   [2./math.sqrt(13),3./math.sqrt(10)]]), np.array([1., 1.]), np.array([1., 4.]), 0.1))

0.4255588466910251


## Cyclical coordinate descent

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:

* As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
* After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to the previous step.

Return weights

The function should accept the following parameters:

* Feature matrix
* Output array
* Initial weights
* L1 penalty
* Tolerance

e.g. in Python:

In [17]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    """
    : type feature_matrix: 2d numpy array
    : type output: 1d numpy array
    : type initial_weights: 1d numpy array
    : type l1_penalty: float
    : type tolerance: float
    : rtype weights: 1d numpy array
    """
    weights = initial_weights
    len_w = len(weights)
    j = 1
    while True:
        max_delta = 0
        for i in range(len_w):
            old_weight = weights[i]
            new_weight = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            weights[i] = new_weight
            delta_i = abs(new_weight - old_weight)
            if delta_i > max_delta:
                max_delta = delta_i
        if j % 10 == 0 and j // 10 > 0:
            print('round %d, max_delta is %.2f' % (j, max_delta))
        if max_delta < tolerance:
            break
        j = j + 1
    return weights

Let us now go back to the simple model with 2 features: `sqft_living` and `bedrooms`. Using `get_numpy_data` (or equivalent), extract the feature matrix and the output array from the house dataframe. Then normalize the feature matrix using `normalized_features()` function.

Using the following parameters, learn the weights on the sales dataset.

* Initial weights = all zeros
* L1 penalty = 1e7
* Tolerance = 1.0

**Quiz Question 3**: What is the RSS of the learned model on the normalized dataset?

In [18]:
feature_matrix, output_array = get_numpy_data(sales, ['sqft_living', 'bedrooms'], ['price'])
normalized_features, norms = normalize_features(feature_matrix)

weights_qz3 = lasso_cyclical_coordinate_descent(
    normalized_features, 
    output_array[:, 0], 
    np.array([0, 0, 0]), 
    1e+7, 
    1.0)

round 10, max_delta is 2266212.32
round 20, max_delta is 381663.88
round 30, max_delta is 64278.04
round 40, max_delta is 10825.01
round 50, max_delta is 1822.21
round 60, max_delta is 306.88
round 70, max_delta is 50.94
round 80, max_delta is 8.49
round 90, max_delta is 1.43


In [19]:
err_qz3 = predict_output(normalized_features, weights_qz3) - output_array[:, 0]
rss_qz3 = np.dot(err_qz3, err_qz3)
print('RSS is %.2E' % rss_qz3)

RSS is 1.63E+15


**Quiz Question 4**: Which features had weight zero at convergence?

In [20]:
all_features_qz3 = np.array(['constant', 'sqft_living', 'bedrooms'])
all_features_qz3[weights_qz3 == 0]

array(['bedrooms'], dtype='<U11')

## Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets. If you are using GraphLab Create, call `random_split` with `.8` ratio and `seed=0.` Otherwise, please download the corresponding csv files from the downloads section.

Create a normalized feature matrix from the TRAINING data with the following set of features.

* bedrooms, bathrooms, sqft_living, sqft_lot, floors, waterfront, view, condition, grade, sqft_above, sqft_basement, yr_built, yr_renovated

Make sure you store the norms for the normalization, since we’ll use them later.

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`. Call resulting weights `weights1e7`, you will need them later.

**Quiz Question 5**: What features had non-zero weight in this case?

In [21]:
all_features_qz4 = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']

feature_matrix, output_array = get_numpy_data(
    train.astype({'floors': 'float64'}), 
    all_features_qz4, ['price'])

In [22]:
normalized_features, norms = normalize_features(feature_matrix)

In [23]:
weights1e7 = lasso_cyclical_coordinate_descent(
    normalized_features, 
    output_array[:, 0], 
    np.repeat(0, len(all_features_qz4)+1), 
    l1_penalty = 1e+7, 
    tolerance = 1)

round 10, max_delta is 2706151.13
round 20, max_delta is 222753.24
round 30, max_delta is 40345.69
round 40, max_delta is 7307.62
round 50, max_delta is 1323.31
round 60, max_delta is 239.10
round 70, max_delta is 43.29
round 80, max_delta is 8.38
round 90, max_delta is 2.22


In [24]:
all_features_np_qz4 = np.array(['constant'] + all_features_qz4)
all_features_np_qz4

array(['constant', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated'], dtype='<U13')

In [25]:
all_features_np_qz4[weights1e7 != 0]

array(['constant', 'sqft_living', 'waterfront', 'view'], dtype='<U13')

Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`. Call resulting weights `weights1e8`, you will need them later.

**Quiz Question 6**: What features had non-zero weight in this case?

In [26]:
weights1e8 = lasso_cyclical_coordinate_descent(
    normalized_features, 
    output_array[:, 0], 
    np.repeat(0, len(all_features_qz4)+1), 
    l1_penalty = 1e+8, 
    tolerance = 1)

In [27]:
weights1e8

array([71114625,        0,        0,        0,        0,        0,
              0,        0,        0,        0,        0,        0,
              0,        0])

In [28]:
all_features_np_qz5 = np.array(['constant'] + all_features_qz4)
all_features_np_qz5[weights1e8 != 0]

array(['constant'], dtype='<U13')

Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`. Call resulting weights `weights1e4`, you will need them later. (This case will take quite a bit longer to converge than the others above.)

**Quiz Question 7**: What features had non-zero weight in this case?

In [29]:
weights1e4 = lasso_cyclical_coordinate_descent(
    normalized_features, 
    output_array[:, 0], 
    np.repeat(0, len(all_features_qz4)+1), 
    l1_penalty = 1e+4, 
    tolerance = 5e+5)

round 10, max_delta is 1647982.54
round 20, max_delta is 1177722.10
round 30, max_delta is 1092904.48
round 40, max_delta is 965783.06
round 50, max_delta is 832351.92
round 60, max_delta is 717339.56
round 70, max_delta is 623694.08
round 80, max_delta is 552427.87
round 90, max_delta is 497885.44


In [30]:
weights1e4

array([ 78564648, -22097380,  12791055,  93808080,  -2013171,  -4219159,
         6482842,   7127409,   5001681,  14327499, -15770951,  -5159586,
       -84495278,   2824439])

In [31]:
all_features_np_qz6 = np.array(['constant'] + all_features_qz4)
all_features_np_qz6[weights1e4 != 0]

array(['constant', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated'], dtype='<U13')

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights. To use these weights on a test set, we must normalize the test data in the same way. Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data:

In this case, we must scale the resulting weights so that we can make predictions with original features:

* Store the norms of the original features to a vector called `norms`:

`features, norms = normalize_features(features)`

* Run Lasso on the normalized features and obtain a `weights` vector

* Compute the weights for the original features by performing element-wise division, i.e.

`weights_normalized = weights / norms`

Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`). To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then

`print(normalized_weights1e7[3])`

should print `161.31745624837794`.

In [32]:
normalized_weights1e4 = weights1e4 / norms
normalized_weights1e7 = weights1e7 / norms
normalized_weights1e8 = weights1e8 / norms

In [33]:
print(normalized_weights1e7[3])

161.31746840896935


## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data. Extract the feature matrix and output array from the **TEST** set. But this time, do **NOT** normalize the feature matrix. Instead, use the normalized version of weights to make predictions.

Compute the RSS of each of the three normalized weights on the (unnormalized) feature matrix.

In [34]:
feature_matrix_test, output_array_test = get_numpy_data(
    test.astype({'floors': 'float64'}), 
    all_features_qz4, ['price'])

rss_weights1e4 = np.sum((predict_output(feature_matrix_test, normalized_weights1e4) - output_array_test[:, 0]) ** 2)
rss_weights1e7 = np.sum((predict_output(feature_matrix_test, normalized_weights1e7) - output_array_test[:, 0]) ** 2)
rss_weights1e8 = np.sum((predict_output(feature_matrix_test, normalized_weights1e8) - output_array_test[:, 0]) ** 2)

print('rss_weights1e4: %.2E' % rss_weights1e4)
print('rss_weights1e7: %.2E' % rss_weights1e7)
print('rss_weights1e8: %.2E' % rss_weights1e8)

rss_weights1e4: 2.28E+14
rss_weights1e7: 2.76E+14
rss_weights1e8: 5.37E+14


**Quiz Question 8**: Which model performed best on the test data?

weights1e4